![book header](pictures/header.png)

[Table of Contents](0_Table_of_Contents.ipynb)

# Chapter 5: Module 3 - Locating KITT Using Audio Communication

**Contents:**
* [Pre-recorded Data](#pre-recorded-data)
* [Background Knowledge](#background-knowledge)
* [Step 1: Implementing Channel Estimation ](#step-1-implementing-channel-estimation)
* [Step 2: Determining Time of Arrivals](#step-2-determining-time-of-arrivals)
* [Step 3: Pulse Segmentation](#step-3-pulse-segmentation)
* [Step 4: Calculating TDOA Between Microphone Pairs](#step-4-calculating-tdoa-between-microphone-pairs)
* [Sanity Check](#sanity-check)
* [Localization Using TDOA Information](#localization-using-tdoa-information)
* [Localization Class](#localization-class)
* [Optional Extension](#optional-extensions)
* [Assessment and Reporting](#assessment-and-reporting)


In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt           # For plotting purposes
import numpy as np                        # For convolution function
from scipy.io import wavfile
from scipy.signal import find_peaks
import time
from scipy.fft import fft, ifft

# Uncomment one of the following lines depending on your setup

# If you are using the real car, uncomment the next lines and comment the simulator lines
# from serial import Serial
# from pyaudio import PyAudio, paInt16

# If you are using the simulator, uncomment the next lines and comment the real car lines
from KITT_Simulator.serial_simulator import Serial
from KITT_Simulator.py_audio_simulator import PyAudio, paInt16

# Note: After changing the import statement, you need to restart the kernel for changes to take effect.

KITT must be located in its field, and directions must be determined to navigate to the final destination. In previous modules, your colleagues have developed scripts to communicate with KITT and read sensor data. You will focus on locating KITT using audio signals received by microphones placed around the field. It is recommended that you read Modules 1 and 2 and talk to the other sub-group frequently. Otherwise the odds of your codes working together are small.

For localization, we will use recordings of the beacon signal at various microphones, deconvolve these using a reference signal, and determine the relative time delays from the resulting channel estimates. These time differences, known as Time Difference of Arrival (TDOA), can be used to estimate KITT's position within the field.

At the end of this module, you will have developed a script to locate KITT within the field with reasonable accuracy, using the data recorded by the microphones. You will also have tested and verified the accuracy and robustness of your solution. 

The module is divided into the following 3 sections:
1. _TDOA Estimation_: In this section, you will estimate the TDOA between pairs of microphones using the provided recordings.
2. _Localization_: In this section, you will use the estimated TDOAs to locate KITT within the field.
3. _Deployment_: In this section, you will adapt the code to work on the real car.

## Pre-recorded Data

To get you started, we provide you with 7 recordings at known locations and a reference recording taken close to one of the microphones. These can be used to develop and test your algorithms. The recordings are at locations randomly distributed across the field, as follows:

| Recording Index |   x [cm]   |   y [cm]   |
|-----------------|------------|------------|
| 0               |    64      |    40      |
| 1               |    82      |   399      |
| 2               |   109      |    76      |
| 3               |   143      |   296      |
| 4               |   150      |   185      |
| 5               |   178      |   439      |
| 6               |   232      |   275      |

*Table 1: Locations of the given recordings (in cm)*

The x and y axes of the field are defined as follows, where the numbers refer to the microphone index:

<img src="pictures/axisdef.png" alt="Field axis and microphone index definition" width="250px">

*Figure: Field axis and microphone index definition*

You can assume these positions for the microphones (note the different height of microphone 5):

| Microphone |   x [cm]   |  y [cm] |  z [cm] |
|------------|------------|---------|---------|
| 1          |     0      |    0    |    50   |
| 2          |     0      |   460   |    50   |
| 3          |    460     |   460   |    50   |
| 4          |    460     |    0    |    50   |
| 5          |     0      |   230   |    80   |

*Table 2: Locations of the microphones (in cm)*

### Loading and Plotting the Provided Recordings

The code below helps you load and plot the first of the 7 audio signals provided. 

In [ ]:
# Coordinates of the recordings
record_x = [64, 82, 109, 143, 150, 178, 232]
record_y = [40, 399, 76, 296, 185, 439, 275]

# List to store filenames
filenames = []

# Generate filenames based on coordinates
for i in range(len(record_x)):
    real_x = record_x[i]
    real_y = record_y[i]
    filenames.append(f"Files/Student Recordings/record_x{real_x}_y{real_y}.wav")

# Load the first recording
Fs, recording = wavfile.read(filenames[0])

# Plot the first channel of the first recording
plt.plot(recording[:, 0])
plt.title(f"Recording at x={record_x[0]} cm, y={record_y[0]} cm, Microphone 1")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

Play around with the recordings and try to understand the data you are working with. Try the following:
- Load and plot another recording.
- Zoom in on one of the pulses.
- Of one pulse, overlay the different microphones. Can you spot the time differences? Can you determine in which corner of the field the recording was made?

In [ ]:
### Student Version ###


*Hints:*
- You can select the recording by changing the index in: ```Fs, recording = wavfile.read(filenames[index])```.
- You can select the microphones by changing the index in: ```recording[:, mic]```.
- The recordings are sampled at 44.1 kHz, but your recordings may have a different sampling rate, depending on what sound card you use. You can check the sampling rate of your recording by printing the variable `Fs`.

## Background Knowledge

In the first week assignments, you developed algorithms for channel estimation using signals received at two microphones. In this module, we extend this to 5 microphones and use this to locate the car.

### Channel Estimation

The channel estimation problem is as follows:

Suppose we transmit a known signal \( x[n] \) over a communication channel and measure the received signal \( y[n] \). The channel acts as a filter, which we will assume to be linear and time-invariant (LTI). Therefore, the received signal is a convolution of the transmitted signal by the channel impulse response \( h[n] \):

$$
y[n] = x[n] * h[n]
$$

Knowing the transmitted signal $x[n]$, can we recover the impulse response of the communication channel $h[n]$ from $y[n]$? This is essentially an inversion problem.

#### Channel Estimation Algorithms

There are several methods for channel estimation:

1. **Deconvolution in Time Domain (ch1):** Involves matrix inversion. It is computationally complex and requires lots of memory.

2. **Matched Filter (ch2):** Avoids matrix inversion by computing the cross-correlation of $y[n]$ with $x[n]$. The channel estimate is given by:

$$
\hat{h}[n] = y[n] * x[-n]
$$

This method depends heavily on having a reference signal with good autocorrelation properties: $r[n] = x[n] * x[-n]$ should resemble a delta-spike.

3. **Deconvolution in Frequency Domain (ch3):** Involves using the Fast Fourier Transform (FFT). Convolution in time becomes multiplication in frequency, so deconvolution becomes division in the frequency domain:

$$
\hat{H}[k] = \frac{Y[k]}{X[k]}
$$

Then, the channel impulse response $ \hat{h}[n] $ is obtained by taking the inverse FFT of $ \hat{H}[k] $.

This method is efficient but requires handling divisions by zero or very small values, which can be achieved by thresholding. Apart from edge effects, the result should be identical to that of **ch1**.

**Note:** We recommend using method **ch3** (Deconvolution in Frequency Domain) for this module.

### Time Difference of Arrival (TDOA)

After estimating the channel impulse response for each microphone, we can detect the first incoming path (the direct path from KITT to the microphone). This corresponds to the propagation delay of the car beacon to each microphone. However, since we do not know the exact transmission time, we can only obtain relative propagation delays. By taking the differences of these delays between pairs of microphones, we eliminate the unknown transmission time, resulting in Time Difference of Arrival (TDOA) measurements.

These TDOA measurements can be used to estimate the position of KITT in the field.


## Step 1: Implementing Channel Estimation

Using the provided reference signal and your channel estimation algorithm (ch3: deconvolution in the frequency domain), deconvolve the recordings to obtain the channel impulse response for each microphone.

#### Steps:

1. **Load and Normalize the Recorded and Reference Signal:**

   - The reference signal is provided as a WAV file recorded close to one of the microphones.
   - Normalize the signals to have a maximum amplitude of 1. This is important because the amplitude of the received signals may vary depending on the microphone.

2. **Segment the Signals:**

   - Manually extract a single pulse from the reference signal and the recordings.

3. **Implement the Channel Estimation Algorithm:**

   - Use ch3 to estimate the channel impulse response.
   - Be sure to handle divisions by zero or very small values in the frequency domain by applying a threshold.

**Notes and Hints:**

- **Normalization:** You can normalize a signal by dividing it by its maximum absolute value:

  ```python
  ref_signal = ref_signal / np.max(np.abs(ref_signal))
  ```

- **FFT Length:** When performing convolution or deconvolution via FFT, ensure that the FFT length is sufficient to avoid "circular convolution" effects (you will learn about this in EE3S1). Calculate `N = len(ref_pulse) + len(rec_pulse) - 1`, and zero-pad both ref_pulse and rec_pulse to length $N$ before doing the FFT. This ensures that circular convolution becomes linear convolution.

- **Thresholding:** Adjust the threshold value based on the magnitude of the FFT of the reference signal. Experiment with different threshold values to find one that works well, starting with a small value (e.g., 1e-6).

In [ ]:
### Student Version ###

# Load and normalize the reference signal
Fs_ref, ref_signal = wavfile.read("Files/Student Recordings/reference.wav")
ref_signal = ref_signal[221000:222500, 0]  # Use only one channel
# TODO: Normalize the reference signal

# Plot the reference signal
plt.figure()
plt.plot(ref_signal)
plt.title("Reference Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
### Student Version ###

# TODO: Load a recorded signal
# TODO: Select one channel
# TODO: Normalize the recording

# Plot the recorded signal
plt.figure()
plt.plot(recording)
plt.title("Recorded Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

# TODO: Segment the recording to extract one pulse

# Plot the segmented reference signal
plt.figure()
plt.plot(recording_pulse)
plt.title("Segmented Signal")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
### Student Version ###

# TODO: Implement the channel estimation algorithm (deconvolution in frequency domain)
def channel3(ref_pulse, rec_pulse):
    pass

# Estimate the channel impulse response

h_est = channel3(ref_signal, recording_pulse)

# Plot the estimated channel impulse response
plt.figure()
plt.plot(h_est)
plt.title("Estimated Channel Impulse Response")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.show()

## Step 2: Determining Time of Arrivals
The goal is to get the time differences of arrival (TDOA) between the microphones. As an intermediate step, we will determine the time of arrivals (TOAs) for each microphone. From the peaks in the channel impulse responses, determine the TOAs and store these in a table.

#### Steps:

1. **Identify the Peaks in the Channel Impulse Responses:**

   - The first significant peak corresponds to the direct path from KITT to the microphone.

2. **Calculate the TOAs:**

   - Convert the sample indices to time values using the sampling frequency.

3. **Store the TOAs:**

   - Collect the TOAs for each microphone and store them in a structured format (e.g., a dictionary or table).

**Notes and Hints:**

- The index of the maximum value in the channel impulse response can be found using:

  ```python
  peak_index = np.argmax(np.abs(h_est))
  ```

- Remember that the absolute time of transmission is unknown, so the TOAs are relative to the arbitrary start of your recording.

In [ ]:
### Student Version ###
# TODO: Identify the peak in the channel impulse response

# TODO: Calculate the TOA (in seconds)

# TODO:Store the TOA for the microphone

# TODO: Print the TOA

## Step 3: Pulse Segmentation

Uptil now, you have manually selected a single pulse from the recordings. In practice, you will need to automate this process. Implement a pulse detection algorithm to automatically segment the pulses from the recordings. We leave this part open to you, but we provide some hints below. 

**Notes and Hints:**

- **Pulse Detection:**

  - `find_peaks` from `scipy.signal` can be used to detect peaks in the envelope of the signal.
  - Adjust the `height` and `distance` parameters based on the characteristics of your signal.

- **Outlier Rejection:**

  - You may need to reject some detected pulses that give a very different TOA compared to the other segments.

- **Single Pulse Importance:**

  - You may be tempted to compute the channel response for the whole recording. However, the computational complexity of this approach is high. Instead, focus on a single pulse.

In [ ]:
### Student Version ###

# TODO: Develop a function to detect the start indices of pulses in the recording
# Takes signal and number of pulses in the recordings as input and returns a list of start indices
def detect_pulses(signal, num_pulses):
    return peaks

# TODO: Load the recording

# Detect pulses in the recorded signal
pulse_starts = detect_pulses(recording_channel, num_pulses)

plt.plot(recording_channel)
plt.plot(pulse_starts, recording_channel[pulse_starts], 'x')
plt.show()

In [ ]:
### Student Version ###

# TODO: Develop a function that takes in a recording and returns the TOA for each microphone
def estimate_TOAs(recording, num_pulses):
    TOAs = {}
    return TOAs

Fs, recording = wavfile.read(filenames[1])
print(estimate_TOAs(recording, 5))

## Step 4: Calculating TDOA Between Microphone Pairs

Complete the TDOA function to calculate the time difference of arrival between pairs of microphones.

#### Steps:

1. **Calculate TDOA Between Two Microphones:**

   - For each pair of microphones, calculate the difference in TOAs.

2. **Convert TDOA to Distance Difference:**

   - Use the speed of sound (approximately 343 m/s) to convert TDOA to distance difference.

3. **Store and Use TDOA Values:**

   - Collect TDOA values for all relevant microphone pairs.

In [ ]:
### Student Version ###

# TODO: Implement a function to calculate TDOA between two microphones
def calculate_tdoa(TOA1, TOA2):
    pass
    return tdoa

# Load the TOAs for each microphone
Fs, recording = wavfile.read(filenames[1])
TOAs = estimate_TOAs(recording, 5)

# TODO: Calculate TDOA between Mic1 and other microphones

# Print the results
print(f"TDOA between Mic1 and Mic2: {tdoa_12} seconds")

print(f"TDOA between Mic1 and Mic3: {tdoa_13} seconds")

print(f"TDOA between Mic1 and Mic4: {tdoa_14} seconds")

print(f"TDOA between Mic1 and Mic5: {tdoa_15} seconds")

## Sanity Check
The accuracy of the TDOA measurements is crucial for the localization accuracy. You can test the accuracy of your TDOA estimation by calculating the TDOA based on the known recording positions and comparing it with the estimated TDOA.

Develop a test function using Pythagoras’ theorem that takes an $(x, y)$ position as input and calculates the theoretical TDOAs you would observe from this position for all microphone pairs. This involves calculating the distance from the given point to each microphone.

This will help you:

- Debug your TDOA function.
- Verify your position estimation in the next steps.

In [ ]:
### Student Version ###
# Microphone positions (x, y, z) in meters
microphone_positions = {
    1: np.array([0.0, 0.0, 0.50]),
    2: np.array([0.0, 4.60, 0.50]),
    3: np.array([4.60, 4.60, 0.50]),
    4: np.array([4.60, 0.0, 0.50]),
    5: np.array([0.0, 2.30, 0.80])
}

#TODO: Function to calculate theoretical TDOAs from a given (x, y) position
def calculate_theoretical_tdoas(car_position, mic_positions):
    return D12, D13, D14, D15

# Example usage:
# Car position in meters
car_position = np.array([1.43, 2.96])  # Replace with desired (x, y) position

print(calculate_theoretical_tdoas(car_position, microphone_positions))

## Localization Using TDOA Information

Now we arrive at the next question studied in this module: **How can we locate the car using the TDOA estimates?** With 5 microphones, we can compute the TDOAs between all pairs of microphones and obtain 10 TDOA measurements. Next, you need an algorithm to convert these TDOA measurements into the $(x, y)$ location of the car.

We offer two approaches for this:

1. **Linear Algebra Approach:** Study [Appendix C](appendix/Appendix_C.ipynb), which shows a basic algorithm to solve for $(x, y)$ using linear algebra. This algorithm is sub-optimal but should be relatively fast.

2. **Grid Search Method:** Perform a grid search over possible $(x, y)$ positions and evaluate which position best matches the estimated TDOAs. This method can be accurate but may be slower depending on implementation.

Implement one of these methods inside the coordintate_2d function.

Although we compute the $(x, y)$ position, in reality, the world is 3D, and the microphones have a certain height above the height of the audio beacon (we define the beacon to sit at $z=0$). The algorithm in Appendix C does not take this height into account. However, this extension is straightforward and you should add it.

In [ ]:
### Student Version ###
# Implement a function to estimate the 2D coordinates of the car
def coordinate_2d(D12, D13, D14, D15):
    xyMic = np.array([[0, 0, 50], [0, 460, 50], [460, 460, 50], [460, 0, 50], [0, 230, 80]])
    return x, y

## Localization Class

Now, it's time to start setting up a processing pipeline by putting everything together. You will develop a `Localization` class that takes a 5-channel recording as input and returns the $(x, y)$ coordinates of the car.

### Suggested Class Structure

**Notes:**

- **Modularity:** Separates different stages of processing for clarity and reusability.
- **Extensibility:** You can add more methods or attributes as needed.
- **Implementation:** You need to implement the `estimate_tdoas` method based on your earlier code.

In [ ]:
### Student Version ###

from scipy.io import wavfile
import logging
import scipy.signal as signal
from scipy.fft import fft, ifft
import numpy as np
import matplotlib.pyplot as plt

class Localization:
    """
    The Localization class processes multi-microphone audio recordings to estimate the position of a sound source (e.g., KITT) using Time Difference of Arrival (TDOA) measurements.

    Attributes:
    - recording (numpy.ndarray): A 2D array containing the recordings from multiple microphones. Shape is (num_samples, num_mics).
    - refSignal (numpy.ndarray): The reference signal used for channel estimation.
    - num_pulses (int): The number of pulses present in the recording.
    - Fs (int): The sampling frequency of the recordings.
    - mic_positions (numpy.ndarray): Array containing the positions of the microphones in centimeters.
    - localizations (tuple): Estimated (x, y) position of the sound source.
    """
    def __init__(self, recording, refSignal, num_pulses, Fs):
        """
        Initialize the Localization object with the given recordings and parameters.

        Parameters:
        - recording (numpy.ndarray): A 2D numpy array of shape (num_samples, num_mics) containing the recordings from multiple microphones. Each column corresponds to a microphone.
        - refSignal (numpy.ndarray): A 1D numpy array containing the reference signal used for channel estimation.
        - num_pulses (int): The number of pulses in the recording.
        - Fs (int): The sampling frequency of the recordings in Hz.
        """

    def localization(self):
        """
        Process the recordings to estimate the 2D position of the sound source.

        This method performs the following steps:
        - Segments the recordings into individual pulses based on the number of pulses.
        - For each pulse:
            - Extracts the pulse segment from the recording.
            - Calculates the Time Difference of Arrival (TDOA) between microphone pairs using the extracted pulse.
        - Averages the TDOA measurements across all pulses.
        - Converts the averaged TDOAs to distance differences.
        - Estimates the 2D coordinates of the sound source using the TDOA measurements.

        Returns:
        - x_car (float): Estimated x-coordinate of the sound source in centimeters.
        - y_car (float): Estimated y-coordinate of the sound source in centimeters.
        """
        
        return x_car, y_car

    def coordinate_2d(self, D12, D13, D14):
        """
        Estimate the 2D coordinates of the sound source based on TDOA measurements.

        This method constructs a system of linear equations using the TDOA measurements and microphone positions and solves for the (x, y) coordinates of the sound source.

        Parameters:
        - D12 (float): Distance difference between Microphone 1 and Microphone 2 in centimeters.
        - D13 (float): Distance difference between Microphone 1 and Microphone 3 in centimeters.
        - D14 (float): Distance difference between Microphone 1 and Microphone 4 in centimeters.

        Returns:
        - position (numpy.ndarray): A 1D array containing the estimated x and y coordinates [x, y] in centimeters.
        """

        return x, y  # Return the estimated (x, y) coordinates
    
    def previously_implemented_methods(self):
        # Copy paste the previously implemented methods
        pass

if __name__ == "__main__":
    # TODO: Load the recording

    # TODO: Load the reference signal

    # TODO: Initialize the Localization object
    
    # TODO: Get the localized position

    print(f"Estimated position: x={x_car}, y={y_car}")


## Optional Extensions

If you finish the basic assignment quickly and want to challenge yourself further, try adding additional functionality to your program. For example:

### 1. Accounting for Height Differences

The current set of linear equations does not consider the height difference between the microphones and the car. This leads to slight inaccuracies, especially when the car is close to a microphone.

**Extension Task:**

- Augment the equations to include the $z$ variable (height of the car).
- Use the known value of $z_{\text{car}} = 0$ to eliminate the variable or adjust the equations accordingly.

### 2. Implementing Advanced Localization Algorithms

The provided method is simple but may be unreliable in certain situations (e.g., when the distances to two microphones are equal).

**Extension Task:**

- Research and implement more advanced algorithms from literature, such as:

    - Stephen Bancroft, “An algebraic solution of the GPS equations”, IEEE Transactions on Aerospace and Electronic Systems, vol.21, no.7, pp.56-59, January 1985.
    - Amir Beck, Petre Stoica, and Jian Li, “Exact and Approximate Solutions of Source Localization Problems”, IEEE Transactions on Signal Processing, vol.56, no.5, pp. 1770-1778, May 2008.

### 3. Grid Search Method

Instead of the linear algebra approach, implement a grid search where the room is partitioned into a dense grid of possible positions, and each location is evaluated against the TDOA data to find the best fit.

**Extension Task:**

- Perform a coarse grid search with larger steps (e.g., 10 cm).
- Refine the search in regions with the best matches.

**Hints:**

- Calculate the theoretical TDOAs for each grid point.
- Compute an error metric (e.g., sum of squared differences) between theoretical and measured TDOAs.
- Select the grid point with the minimum error as the estimated position.


## Assessment and Reporting

### Mid-term Assessment and Report

Ultimately in Week 4, you will showcase the functionality of your localization script to your assigned TA. You should demonstrate proper localization of the car on the 3 recordings with unknown coordinates.

After you pass this assessment, you are ready to document your results in your midterm report. A detailed report is required, covering:

- **Approach:** Describe the methods and algorithms used.
- **Implementation:** Explain how you implemented the algorithms.
- **Testing:** Show results from tests on simulated and provided test data.
- **Results:** Present localization results, error analysis, and accuracy.
- **Conclusion:** Summarize the expected accuracy and reliability of your localization method.

Please review the guidelines in Chapter 7 for more information.

### After the Midterm: Reference Signal and Integration

If you have completed this module successfully, you can start integrating and estimating the car’s location from your own microphone measurements.

#### Reference Signal Selection

Using a good beacon signal is important, because it determines the quality of your location estimates. Since the time in IP3 is limited, we will give a few general hints, but note that the topic could be explored in much more detail.

In your ch3 algorithm (deconvolution in frequency domain), you have seen that we omit all frequencies where the beacon signal amplitude spectrum $|X(F)|$ is small, because we divide by $X(F)$ and this would blow up the noise. For best results, we want $X(F)$ to be large for all frequencies. Therefore, ideally, we have a flat power spectrum. For random signals, this corresponds to the use of "white noise".   

**Beacon code** Our beacon code consists of 32 bits, each 0 or 1. White noise corresponds to a beacon code for which the autocorrelation resembles a delta spike. You could generate a bit code using 'rand', and then round to 0 or 1. A further consideration is that the code should start and end with '1', or else you are using in fact a code that is shorter than 32 bits. 

After generating a code, check its autocorrelation function, $x[n]\ast x[-n]$. 
You want a strong peak for the 0-lag of the autocorrelation but as low as possible for any other lag.
*Hint:* Randomly generated codes are suitable for our purposes. You could try some optimal codes (check communication theory literature for “gold codes”).

**Carrier frequency and bit frequency** The carrier frequency defines the "pitch" of the transmitted signal. Since we use audio equipment which is optimized for human listening, it is better to use carriers below 10 kHz. 

The bit frequency determines the bandwidth of the transmitted signal. Since we want to probe the channel on as many different frequencies as possible, we would select a large bit frequency (but it can't be larger than the carrier frequency). The hardware implementation seems to limit the total output power. If we use a wider bandwidth, the energy per hertz is reduced. Thus, there is a trade-off. You can try to find good values experimentally, i.e. try a range of bit frequencies in steps of 1000 Hz, and see which setting gives most accurate results.

A perfect repetition count is not yet required; but you need to make sure that the full code can be transmitted and recorded on all microphones within the same recording window.

**Field Testing:** A theoretical signal may not work well in practice. Use your field test time well. Consider recording a bunch of different audio beacon settings and analyzing them later.

### Integration Assignment

#### **Make Test Recording**

- Record the signal transmitted over KITT’s beacon with your selected parameters.
- Place the microphone close to KITT’s beacon to get a clean recording.
- Avoid clipping by adjusting recording levels.

#### **Create Reference Signal**

- Load the recording into Python.
- Clean it by removing zero intervals and extracting a single pulse.
- Use this as your reference signal in deconvolution.

#### **Test Performance**

- Place KITT at a known location (e.g., the center of the field).
- Make recordings and run your localization algorithm.
- Evaluate the accuracy.

#### **Integrate with Recording Code**

- Combine your localization algorithm with the recording code.
- Address any issues such as blocking recording.
- Aim to locate KITT in real-time while it is moving.

#### **Optional Refinement**

- Add timestamp information to your location estimates.
- Consider the delays between recording, processing, and KITT's movement.

### Deliverable (Final Report)

Show your selected beacon parameters and comment on the resulting performance.

- **Accuracy:** How accurate is your localization algorithm?
- **Real-time Operation:** Are you able to drive and locate KITT simultaneously in real-time?
- **Discussion:** Address any challenges faced and how you overcame them.

**Note to Students:**

- Remember to document your code thoroughly.
- Include explanations for your choices and any assumptions made.
- Test your algorithms extensively with both simulated and real data.
